<div style="display: flex; align-items: center;">
    <img alt="udeA logo" height="140px" src="https://github.com/freddyduitama/images/blob/master/logo.png?raw=true" style="width:107px;height:152px; margin-right: 40px;">
    <div style="text-align: center;">
        <h1><b><font color='0B5345'>Proyecto 4: Clasificación de señales ECG</font></b></h1>
        <h2><font color='0B5345'>Proyecto 2024-02</font></h2>
        <h3><font color='0B5345'>Luisa Fernanda Enciso - Jader Chingal</font></h3>
    </div>
</div>

In [3]:
import neurokit2 as nk
import pandas as pd
import seaborn as sns
import scipy.signal as signal
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import random
from scipy.signal import butter, filtfilt
import pywt
from scipy.signal import detrend
import warnings
from scipy.signal import welch
from sklearn.metrics import mean_squared_error
from skimage.restoration import denoise_nl_means, estimate_sigma
from scipy.signal import savgol_filter
from statsmodels.nonparametric.smoothers_lowess import lowess
warnings.filterwarnings('ignore')


<hr>

Extracción de los registros que corresponden a bradicardia sinusal (SB Sinus Bradycardia) y fibrilación auricular (AFIB Atrial Fibrillation). 

In [4]:
# Cargar el archivo Excel en un DataFrame
ruta=r"C:\Users\luisa\OneDrive - Universidad de Antioquia\Documentos\BIOSENALES24II\Proyecto2\Diagnostics.xlsx"
diagnostics_df = pd.read_excel(ruta)

# Crear el nuevo dataframe filtrando por la columna 'Rhythm'
datos = diagnostics_df[(diagnostics_df['Rhythm'] == 'SB') | (diagnostics_df['Rhythm'] == 'AFIB')]

# Discriminar datos de acuerdo a su caracteristica 
datos_SB = diagnostics_df[(diagnostics_df['Rhythm'] == 'SB')]
datos_AFIB = diagnostics_df[(diagnostics_df['Rhythm'] == 'AFIB')]

# Convertir la columna 'FileName' en una lista de Python
datos_lista = datos['FileName'].tolist()

datos_lista_SB = datos_SB['FileName'].tolist()
datos_lista_AFIB = datos_AFIB['FileName'].tolist()

datos.head()

,FileName,Rhythm,Beat,PatientAge,Gender,VentricularRate,AtrialRate,QRSDuration,QTInterval,QTCorrected,RAxis,TAxis,QRSCount,QOnset,QOffset,TOffset
0,MUSE_20180113_171327_27000,AFIB,RBBB TWC,85,MALE,117,234,114,356,496,81,-27,19,208,265,386
1,MUSE_20180112_073319_29000,SB,TWC,59,FEMALE,52,52,92,432,401,76,42,8,215,261,431
3,MUSE_20180113_121940_44000,SB,NONE,66,MALE,53,53,96,456,427,34,3,9,219,267,447
5,MUSE_20180112_120347_79000,SB,NONE,46,FEMALE,57,57,70,404,393,38,24,9,225,260,427
6,MUSE_20180114_075026_69000,AFIB,TWC,80,FEMALE,98,86,74,360,459,69,83,17,215,252,395


In [ ]:
target_file = 'MUSE_20180113_171327_27000'
datos_target = diagnostics_df[diagnostics_df['FileName'] == target_file]
print(datos_target)



                     FileName Rhythm      Beat  PatientAge Gender  \
0  MUSE_20180113_171327_27000   AFIB  RBBB TWC          85   MALE   

   VentricularRate  AtrialRate  QRSDuration  QTInterval  QTCorrected  RAxis  \
0              117         234          114         356          496     81   

   TAxis  QRSCount  QOnset  QOffset  TOffset  
0    -27        19     208      265      386  


In [3]:
# Imprimir los tamaños de las listas
print(f"Tamaño total de Datos (SB + AFIB): {len(datos_lista)}")
print(f"Tamaño de (SB): {len(datos_lista_SB)}")
print(f"Tamaño de (AFIB): {len(datos_lista_AFIB)}")

Tamaño total de Datos (SB + AFIB): 5669
Tamaño de (SB): 3889
Tamaño de (AFIB): 1780


<hr>

Seleccionar señales de la base de datos.

In [4]:
# Definir la ruta del archivo donde se guardarán los nombres de archivo seleccionados
archivo_seleccion = r"C:\Users\luisa\OneDrive - Universidad de Antioquia\Documentos\BIOSENALES24II\Proyecto4\seleccion_filename.txt"

# Comprobar si el archivo de selección ya existe
if os.path.exists(archivo_seleccion):
    # Cargar los nombres de archivo previamente guardados
    with open(archivo_seleccion, "r") as f:
        seleccion = f.read().splitlines()
    seleccion_SB = seleccion[:5]
    seleccion_AFIB = seleccion[5:]
else:
    # Si el archivo no existe, realizar una selección aleatoria y guardarla
    seleccion_SB = random.sample(datos_SB['FileName'].tolist(), 5)
    seleccion_AFIB = random.sample(datos_AFIB['FileName'].tolist(), 5)
    
    # Guardar solo los nombres de archivo en el archivo de texto
    with open(archivo_seleccion, "w") as f:
        for filename in seleccion_SB + seleccion_AFIB:
            f.write(f"{filename}\n")

# Imprimir las selecciones
print("Señales seleccionadas de SB:", seleccion_SB)
print("Señales seleccionadas de AFIB:", seleccion_AFIB)

Señales seleccionadas de SB: ['MUSE_20180112_072624_20000', 'MUSE_20180118_132206_33000', 'MUSE_20180118_132819_96000', 'MUSE_20180112_072725_42000', 'MUSE_20180114_135040_64000']
Señales seleccionadas de AFIB: ['MUSE_20180712_160002_41000', 'MUSE_20180114_122956_07000', 'MUSE_20180119_170822_29000', 'MUSE_20180120_121731_93000', 'MUSE_20180113_131312_49000']


<hr>

Aplicar tutorial de NeuroKit